In [1]:
import pandas as pd
import numpy as np
import os
import glacierml as gl
import matplotlib.pyplot as plt
from tqdm import tqdm
pd.set_option('display.max_columns',None)

2024-01-12 12:23:26.743025: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-12 12:23:26.786603: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-12 12:23:27.355695: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
df = gl.coregister_data('4')
df = df.drop(df[df['RGIId'].duplicated(keep = False) == True].index)
df = df.sample(frac = 1,random_state = 0)
df = df.reset_index().drop('index', axis = 1)
#### Add mean thickness estimates to dataset ####
# model_path = '/data/fast1/glacierml/models/LOO/'
model_path = os.path.join(
    '/data/fast1/glacierml/models/LOO/' ,'rgi_est_raw.pkl'
)
# RGI = pd.read_pickle(model_path + 'rgi_est_raw.pkl') ascendingThickness_rgi_est_raw_.pkl
RGI = pd.read_pickle(model_path) 
rgi_list = list(df)[:-1]
rgi_list.append('RGIId')
old_index = df.index.to_list()
# df = pd.merge(df,RGI,how = 'inner', on = rgi_list)

#### Add Farinotti mean thickness estimates ####
ref_pth = '/data/fast1/glacierml/data/reference_thicknesses/'
ref = pd.DataFrame()
for file in os.listdir(ref_pth):
    if 'Farinotti' in file:
        file_reader = pd.read_csv('reference_thicknesses/' + file)
        ref = pd.concat([ref, file_reader], ignore_index = True) 
ref = ref.rename(columns = {
     'Farinotti Mean Thickness':'FMT',
})
ref = ref[[
     'FMT',
     'RGIId',
]]

# df = pd.merge(df, ref, how = 'inner', on = 'RGIId')
df = pd.merge(RGI, ref, how = 'inner', on = 'RGIId')
# cols = df.sort_values('Thickness', ascending = True).index
cols = []
for i in range(273):
    cols.append(i)
# df = df.sort_values('Thickness', ascending = True)
data = df
data['est'] = np.mean(df[cols],axis = 1)
# data = data.dropna(subset = 'FMT')

/tmp/ipykernel_66811/1085198024.py:40: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data['est'] = np.mean(df[cols],axis = 1)


In [3]:
data

,RGIId,GLIMSId,BgnDate,EndDate,CenLon,CenLat,O1Region,O2Region,Area,Zmin,Zmax,Zmed,Slope,Aspect,Lmax,Status,Connect,Form,TermType,Surging,Linkages,Name,region,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,FMT,est
0,RGI60-01.00001,G213177E63689N,20090703,-9999999,-146.8230,63.6890,1,2,0.360,1936,2725,2385,42.0,346,839,0,0,0,0,9,9,NaN,01,16.277309,14.953861,16.694000,16.503345,16.270660,16.387028,16.016808,16.492792,13.731221,15.645153,16.584284,17.033907,16.441111,16.247591,15.764664,17.265984,18.296114,17.446587,17.781895,15.735214,16.638071,17.323732,16.754688,16.614246,18.158390,19.007336,16.249086,17.789362,17.527275,16.842936,17.503490,18.452667,17.554615,17.019976,15.990520,15.349936,18.239737,13.840107,14.836700,18.431520,17.526781,13.500299,16.043545,17.986980,17.532366,16.077944,16.615873,16.622864,18.498369,25.100864,14.253600,15.265696,14.689593,17.423824,12.993851,15.165100,14.188037,14.914692,19.073793,21.630016,13.054076,15.582692,15.758699,25.390738,20.387331,23.640579,18.952639,21.204344,19.756466,23.073219,18.495457,21.120855,20.512154,19.207048,19.453024,17.356770,17.547823,15.365030,15.656080,17.515306,16.157310,15.822584,16.130745,15.710287,15.627931,20.446331,17.713467,17.581598,14.921235,17.123339,18.198267,18.697874,26.132809,16.916780,16.243801,16.174265,14.198225,17.368719,14.767431,19.973389,17.080496,16.496000,16.075808,16.526133,25.402103,25.848440,15.389005,16.136736,16.740885,16.189621,17.583885,25.984201,17.404371,23.669857,13.357930,18.179487,12.855234,11.240031,17.296543,19.448481,15.156308,18.037245,16.257404,16.105001,20.486477,15.540879,15.283449,15.516177,15.282767,19.823141,20.566320,15.502954,19.014490,21.546610,15.918286,17.977177,19.857008,16.855869,18.062645,18.275928,18.532713,17.624506,19.784925,16.665798,11.477992,15.127000,15.541538,17.168047,12.050786,15.876202,15.085879,12.467257,14.309031,11.930584,11.343918,14.936225,10.198374,16.696497,14.893176,10.165212,8.739060,13.874270,9.754492,8.196445,8.045223,8.362290,16.227907,18.265079,16.520351,13.563940,18.026171,10.988451,10.115927,15.983336,9.324627,6.848248,15.672099,16.868109,19.921408,17.147453,15.378146,15.809608,14.955585,15.070559,15.497086,15.061805,17.551743,16.039452,22.005207,16.781631,16.186377,21.432890,16.758854,14.632924,19.698366,17.936598,20.195055,21.196356,18.484770,21.393030,19.014027,14.088100,17.732983,12.879424,13.795535,10.666667,13.161463,22.818604,13.175793,11.826213,12.724860,13.561060,13.090656,13.046973,13.562998,11.996654,14.737124,14.423245,11.989023,14.179888,14.271254,14.572264,13.985809,14.578692,13.800107,11.057722,13.400297,14.671399,14.059649,14.415533,12.463089,14.190796,11.478177,14.395720,13.810367,13.382660,13.147652,14.228340,14.710816,13.920391,15.104506,13.774546,12.476313,13.384902,12.551325,13.021507,13.818346,12.550982,13.267177,14.472327,11.889599,14.496082,14.688834,12.923995,13.887196,13.625621,13.045137,13.295097,12.778278,12.572529,13.148931,14.493125,13.363637,14.629187,12.288808,14.819217,13.791186,14.931712,13.400538,13.239247,12.570488,13.328512,13.999707,19.

In [8]:
def stats_builder(data,st1 = 1e-5, st2 = 1e4):
    
#     boundary = [(str(st1), str(st2))]
    
    col_list = [ 'Area', 
#                 'Slope','Lmax','Zmin',
#                 'Zmax',
                'est',
#                 'Thick Diff',
                'Vol Diff',
#                'FMT'
               ]
    name = ['mean',
            'median', 'min', 'max','IQR',
            'STD','count'
           ]

    df = pd.DataFrame( columns = col_list, index = name)
#     data = gl.load_notebook_data('4','first')
    
    x = data['FMT'].to_numpy()
    y = data['est'].to_numpy()
    area = data['Area'].to_numpy()
    
    vol_f = (x / 1e3) * area
    vol = (y / 1e3) * area
    
    thick_f = vol_f / area * 1e3
    thick = vol / area * 1e3
    
    vol_diff = vol - vol_f
    data['Thick Diff'] = thick - thick_f
    data['Vol Diff'] = vol_diff
    data = data.iloc[np.where(
        (vol > st1) & (vol < st2)
    )]
    vol_sub = vol[np.where(
        (vol > st1) & (vol < st2)
    )]
    feat_list = col_list
    for i in feat_list:
        dft = data[i]
        upp = np.nanquantile(dft, 0.75)
        low = np.nanquantile(dft, 0.25)
#         print(np.quantile(dft, 0.75))
#         print(np.quantile(dft, 0.25))
        functions = [
            np.round(np.nanmean(dft), 1),
            np.round(np.nanmedian(dft), 1), 
            np.round(np.nanmin(dft), 1),
            np.round(np.nanmax(dft), 1),
            np.round(upp - low, 1),
            np.round(np.nanstd(dft),1),
            len(dft)
        ]
        for n, fn in zip(name, functions):
            df[i].loc[n] = fn
    df = df.rename(columns = {
        'Area':'Area (km$^2$)',
#         'Slope':'Slope (deg)',
#         'Lmax':'Max Length (m)',
#         'Zmin':'Min Elevation (m)',
        'est0':'Est Thick (m)',
        'Vol Diff':'Vol Diff (km$^3$)'
#         'Farinotti Mean Thickness':'Farinotti Thickness'
    })
    df = df.round(decimals = 1)
    return df, data, vol, vol_sub

In [12]:
stats, df, vol, vol_sub = stats_builder(data,1e-1,50)
print(stats.to_latex())
stats

\begin{tabular}{llll}
\toprule
 & Area (km$^2$) & est & Vol Diff (km$^3$) \\
\midrule
mean & 14.500000 & 63.000000 & -0.600000 \\
median & 5.000000 & 56.500000 & -0.000000 \\
min & 0.800000 & 15.500000 & -217.900000 \\
max & 774.600000 & 310.799988 & 18.900000 \\
IQR & 8.100000 & 20.200000 & 0.200000 \\
STD & 31.300000 & 22.799999 & 3.600000 \\
count & 24765 & 24765 & 24765 \\
\bottomrule
\end{tabular}



,Area (km$^2$),est,Vol Diff (km$^3$)
mean,14.5,63.0,-0.6
median,5.0,56.5,-0.0
min,0.8,15.5,-217.9
max,774.6,310.799988,18.9
IQR,8.1,20.2,0.2
STD,31.3,22.799999,3.6
count,24765,24765,24765


In [13]:
df['Area']

3          3.681
4          2.573
5         10.470
9          3.806
11         2.802
           ...  
216474     2.180
216476    26.684
216493    13.951
216495     7.970
216498     4.118
Name: Area, Length: 24765, dtype: float64

In [14]:
df['est']

3          46.550385
4          48.636570
5          67.170479
9          53.423527
11         45.823963
             ...    
216474     65.699814
216476     96.304626
216493    100.063545
216495     77.550270
216498     77.720757
Name: est, Length: 24765, dtype: float32